In [ ]:
## main.py -- sample code to test attack procedure
##
## Copyright (C) 2018, IBM Corp
##                     Chun-Chen Tu <timtu@umich.edu>
##                     PaiShun Ting <paishun@umich.edu>
##                     Pin-Yu Chen <Pin-Yu.Chen@ibm.com>
##
## Licensed under the Apache License, Version 2.0 (the "License");
## you may not use this file except in compliance with the License.
## You may obtain a copy of the License at
##
##     http://www.apache.org/licenses/LICENSE-2.0
##
## Unless required by applicable law or agreed to in writing, software
## distributed under the License is distributed on an "AS IS" BASIS,
## WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
## See the License for the specific language governing permissions and
## limitations under the License.

## (C) 2020 Changes by UvA FACT AI group [Pytorch conversion]

import os
import sys
import numpy as np
import random
import time
from setup_mnist import MNIST, MNISTModel
import torch
import utils as util
from CEM import CEM

b = 9
c_init = 0.1
max_iter=1000

In [ ]:
def main(image_id, arg_max_iter=1000, arg_b=9, arg_init_const=10.0, arg_mode="PN", arg_kappa=0, arg_beta=1e-1, arg_gamma=0):
    #with tf.Session() as sess:
    random.seed(121)
    np.random.seed(1211)
    
    #Load autoencoder and MNIST dataset
    AE_model = util.load_AE("mnist_AE_weights")
    data, model =  MNIST(), MNISTModel(torch.load('models/mnist.pt'))
    
    orig_prob, orig_class, orig_prob_str = util.model_prediction(model, data.test_data[image_id].unsqueeze(0))
    target_label = orig_class
    print("Image:{}, infer label:{}".format(image_id, target_label))
    orig_img, target = util.generate_data(data, image_id, target_label)
    
    #create adversarial image from original image
    attack = CEM(model, arg_mode, AE_model, batch_size=1, learning_rate_init=1e-2, 
                 c_init=c_init, c_steps=b, max_iterations=max_iter, 
                 kappa=arg_kappa, beta=arg_beta, gamma=arg_gamma)

    adv_img = attack.attack(orig_img, target)
    
    #calculate probability classes for adversarial and delta image 
    adv_prob, adv_class, adv_prob_str = util.model_prediction(model, adv_img)
    delta_prob, delta_class, delta_prob_str = util.model_prediction(model, orig_img-adv_img)
    
    INFO = "[INFO]id:{}, kappa:{}, Orig class:{}, Adv class:{}, Delta class: {}, Orig prob:{}, Adv prob:{}, Delta prob:{}".format(image_id, arg_kappa, orig_class, adv_class, delta_class, orig_prob_str, adv_prob_str, delta_prob_str)
    print(INFO)
    
    
    #Save image to Results
    suffix = "id{}_kappa{}_Orig{}_Adv{}_Delta{}".format(image_id, arg_kappa, orig_class, adv_class, delta_class)
    arg_save_dir = "{}_ID{}_Gamma_{}".format(arg_mode, image_id, arg_gamma)
    os.system("mkdir -p Results/{}".format(arg_save_dir))
    util.save_img(orig_img, "Results/{}/Orig_original{}.png".format(arg_save_dir, orig_class))
    util.save_img(adv_img, "Results/{}/Adv_{}.png".format(arg_save_dir, suffix))
    util.save_img(np.absolute(orig_img-adv_img)-0.5, "Results/{}/Delta_{}.png".format(arg_save_dir, suffix))

    sys.stdout.flush()

In [ ]:
main(image_id=2950)